<a href="https://colab.research.google.com/github/chasubeen/ESAA_8th_OB/blob/Week_2/%EB%AA%A8%EB%8D%B8%20%ED%9B%88%EB%A0%A8_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **| 모델 훈련 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch04 연습문제 1, 5, 9, 10
- 개념 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

### **1. 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?**
___


### 답안
- 확률적 경사 하강법(SGD)
- 미니배치 경사 하강법
- 단, SGD는 매 스텝에서 한 개의 샘플만을 선택하기에, 모델이 편향된 학습을 할 수 있다는 위험성이 있음(불안정함)
> 미니배치 경사 하강법이 좀 더 타당해 보인다.

### **2. 배치 경사 하강법을 사용하고 에포크마다 검증 오차를 그래프로 나타내봤습니다. 검증 오차가 일정하게 상승되고 있다면 어떤 일이 일어나고 있는 걸까요? 이 문제를 어떻게 해결할 수 있나요?**
___

- 가정 1) 너무 높은 학습률 -> 알고리즘 발산 가능성
  - train loss로 같이 증가하는 경우
- 가정 2) 과적합
  - 모델이 훈련 set에만 너무 최적화되어 학습됨

### **3. 릿지 회귀를 사용했을 때 훈련 오차가 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $\alpha$를 증가시켜야 할까요 아니면 줄여야 할까요?**
___

- 문제: 높은 편향
  - 모델이 훈련 set에 과소적합됨
> 규제 하이퍼 파라미터인 alpha를 감소시켜야 함

### **4. 다음과 같이 사용해야 하는 이유는?**
___
- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀
- 릿지 회귀 대신 라쏘 회귀
- 라쏘 회귀 대신 엘라스틱넷

1) 규제가 있는 모델이 일반적으로 규제가 없는 모델에 비해 성능이 좋음  

2) 라쏘: l1 penalty, 릿지: l2 penalty  
  - 중요도가 낮은 특성은 없애주는 효과를 줄 수 있음
    - 특성 수가 많은 데이터 학습의 경우 차원의 저주 문제를 어느 정도 해결해 줄 도 있음
  - 몇 개의 특성만이 유용할 것이라는 확신이 들면 라쏘를 사용하면 됨
    - 그게 아니라면 일단 릿지도 해보자.  

3) 엘라스틱넷: 릿지 + 라쏘  
  - 적당히 절충된 모델이라 라쏘가 불규칙하게 튀는 경우를 방지해 줌
  - 라쏘에 가깝게 활용하고 싶다면 l1_penalty를 1에 가깝게 주면 됨


### **추가) 조기 종료를 사용한 배치 경사 하강법으로 소프트맥스 회귀를 구현해보세요(사이킷런은 사용하지 마세요)**


---



In [26]:
### 소프트맥스 함수(사용자 정의 함수)

def softmax(logits):
  exps = np.exp(logits)
  exp_sums = np.sum(exps, axis=1, keepdims=True)
  return exps / exp_sums

In [27]:
## 샘플 데이터 생성
# iris 데이터 활용

import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

# Iris 데이터 로드
iris = datasets.load_iris()
X = iris.data
y = iris.target

# 데이터 분리: 학습, 검증, 테스트 데이터
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [33]:
## 초기 하이퍼파라미터 설정

learning_rate = 0.01 # 학습률
num_epochs = 1000 # 반복 횟수
early_stopping_round = 10 # 조기 종료 횟수
prev_loss = float("inf") # 초기 손실값
patience = 0 # 얼마나 기다려 줄 것인가

In [34]:
## 가중치 초기화

W = np.random.rand(X.shape[1], len(np.unique(y)))

In [35]:
# 검증 손실을 저장할 변수

best_val_loss = float("inf")
best_W = None

In [36]:
for epoch in range(num_epochs):
    ## 예측 계산
    train_scores = np.dot(X_train, W)
    train_probs = softmax(train_scores)

    val_scores = np.dot(X_val, W)
    val_probs = softmax(val_scores)

    ## 손실 계산
    # 크로스 엔트로피 손실 계산
    train_loss = -np.log(train_probs[range(len(X_train)), y_train]).mean()

    val_loss = -np.log(val_probs[range(len(X_val)), y_val]).mean()

    ## 학습 상황 출력
    if epoch % 10 == 0:
        print(f"Epoch {epoch}: Train Loss {train_loss}, Val Loss {val_loss}")

    # Early Stopping Check
    if val_loss >= best_val_loss:
        patience += 1
        if patience >= early_stopping_round:
            print("Early stopping!")
            break
    else:
        patience = 0
        best_val_loss = val_loss
        best_W = W.copy()

    ## 기울기(gradient) 계산
    dscores = train_probs
    dscores[range(len(X_train)), y_train] -= 1
    dscores /= len(X_train)

    ## 가중치 업데이트
    dW = np.dot(X_train.T, dscores)
    W -= learning_rate * dW

Epoch 0: Train Loss 2.666016951770774, Val Loss 2.3009781899248156
Epoch 10: Train Loss 1.6438747405927778, Val Loss 1.6337916720911592
Epoch 20: Train Loss 1.398724480201496, Val Loss 1.494065083190535
Epoch 30: Train Loss 1.3020908032874627, Val Loss 1.3854257450926633
Epoch 40: Train Loss 1.218375718862017, Val Loss 1.2757823558967216
Epoch 50: Train Loss 1.144391434800491, Val Loss 1.177702477275301
Epoch 60: Train Loss 1.079474050826249, Val Loss 1.0921965963818827
Epoch 70: Train Loss 1.0227731489050478, Val Loss 1.0182676188681656
Epoch 80: Train Loss 0.97333505832422, Val Loss 0.9545444234421183
Epoch 90: Train Loss 0.9302009770226145, Val Loss 0.8996211123246904
Epoch 100: Train Loss 0.8924718623342165, Val Loss 0.852181818988229
Epoch 110: Train Loss 0.8593427577560608, Val Loss 0.8110556313209759
Epoch 120: Train Loss 0.8301147074846094, Val Loss 0.7752335820079473
Epoch 130: Train Loss 0.8041927468771852, Val Loss 0.743863977530561
Epoch 140: Train Loss 0.7810766098629794, 

- loss가 잘 감소하는 것을 확인할 수 있음

In [37]:
## 예측
test_scores = np.dot(X_test, best_W)
test_probs = softmax(test_scores)
y_pred = np.argmax(test_probs, axis=1)

## 평가
accuracy = np.mean(y_pred == y_test)
print(f"Test Accuracy: {accuracy}")

Test Accuracy: 0.9666666666666667


### **전체 프로세스**
1. **예측 계산**: 현재의 모델 가중치를 사용하여 학습 데이터의 예측 점수를 계산하고, 소프트맥스 함수를 적용하여 클래스별 확률을 얻음
2. **손실 계산**: 예측과 실제 레이블 간의 차이를 측정하는 손실 함수를 사용하여 학습 데이터/검증 데이터의 손실을 계산
3. **Early Stopping 확인**: 검증 손실이 이전 에포크보다 증가하는 경우, 조기 종료를 고려하여 학습을 조기 종료할 수 있음
4. **기울기(gradient) 계산**: 손실 함수의 기울기(gradient)를 계산하여 모델 파라미터(가중치)를 업데이트
5. **가중치 업데이트**: 기울기를 사용하여 모델의 가중치를 업데이트하여 손실을 최소화
6. **최적 모델 선택**: 검증 손실이 최소일 때의 모델 가중치를 최적 모델로 선택
7. **반복 종료 확인**: 미리 설정한 학습 횟수(`num_epochs`)를 모두 수행하거나 조기 종료 조건을 만족하면 학습을 종료
8. **테스트 평가**: 최적 모델을 사용하여 테스트 데이터에 대한 예측을 수행하고 모델의 최종 성능을 평가